In [1]:
# Define file paths
input_file = '/Volumes/MY_SSD_1TB/Work_PhD/July-August/CEERS_data/Romeo_s_data/Drive_cat/final_matched_catalogue.txt'
output_file = '/Volumes/MY_SSD_1TB/Work_PhD/July-August/CEERS_data/Romeo_s_data/Drive_cat/extracted_columns.txt'

# Open input and output files
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    # Process each line
    for line in infile:
        # Skip empty lines
        if not line.strip():
            continue
            
        # Split the line into columns
        columns = line.split()
        
        # Make sure we have enough columns
        if len(columns) >= 3:
            # Write POINTING and Brenjit_ID to output file
            outfile.write(f"{columns[0]} {columns[2]}\n")

print(f"Extracted columns saved to: {output_file}")

Extracted columns saved to: /Volumes/MY_SSD_1TB/Work_PhD/July-August/CEERS_data/Romeo_s_data/Drive_cat/extracted_columns.txt


In [1]:
print(10**(0.04))

1.096478196143185


In [5]:
diff=1
flux_ratio=10**(diff/2.5)
print(flux_ratio)



2.51188643150958


In [6]:
import numpy as np
from astropy.io import ascii
import os

# Configuration
base_dir = '/Volumes/MY_SSD_1TB/Work_PhD/July-August/CEERS_data/SEP_JWST/Results/'
catalog_subdir = 'catalogue_z7'
output_file = 'brenjit_sources_analysis.txt'

# Brenjit_IDs to process
highlight_ids = {
    'nircam1': [2858, 4010, 6802, 8216, 11572, 10899, 8272],
    'nircam2': [1332, 2034, 5726, 11316],
    'nircam3': [9992],
    'nircam4': [315, 1669, 1843, 9859, 9427, 12415, 11659],
    'nircam5': [4547, 9883, 12083, 12779, 12888],
    'nircam6': [3206],
    'nircam7': [7070, 9437, 9959, 11615, 11754, 3581, 13394, 13743, 14097, 14208],
    'nircam8': [1398, 2080, 2288, 4493, 6986, 8954, 9454, 11712, 14604],
    'nircam9': [2646, 3096, 5572, 9061, 8076],
    'nircam10': [1424, 1935, 7847, 9562, 10415]
}

def read_sextractor_catalog(filepath):
    """Reads a SExtractor catalog file."""
    try:
        data = ascii.read(
            filepath,
            comment='#',
            header_start=None,
            data_start=0,
            names=[
                'NUMBER', 'X_IMAGE', 'Y_IMAGE', 'MAG_AUTO', 'MAGERR_AUTO',
                'MAG_APER', 'MAGERR_APER', 'CLASS_STAR', 'FLUX_AUTO',
                'FLUXERR_AUTO', 'FLUX_APER', 'FLUXERR_APER', 'ALPHA_J2000',
                'DELTA_J2000'
            ]
        )
        with np.errstate(divide='ignore', invalid='ignore'):
            snr = data['FLUX_AUTO'] / data['FLUXERR_AUTO']
        snr[np.isinf(snr) | np.isnan(snr)] = 0
        data['SNR'] = snr
        return data
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

def get_mags(data_dict, filter_list):
    """Helper function to extract magnitudes."""
    mags = {}
    for filt in filter_list:
        mags[filt] = np.nan_to_num(data_dict[filt]['MAG_AUTO'], nan=99.0, posinf=99.0, neginf=99.0)
    return mags

def main():
    """Main function to extract data for all Brenjit_ID sources."""
    
    # Prepare output data
    results = []
    
    print(f"Processing {sum(len(ids) for ids in highlight_ids.values())} Brenjit_ID sources...")
    
    for pointing, ids in highlight_ids.items():
        print(f"\nProcessing {pointing}...")
        
        # Load F115W and F150W catalogs
        catalog_dir = os.path.join(base_dir, pointing, catalog_subdir)
        
        # Load F115W data
        f115w_path = os.path.join(catalog_dir, f"f150dropout_f115w_catalog.cat")
        f115w_data = read_sextractor_catalog(f115w_path)
        if f115w_data is None:
            print(f"  Missing F115W data for {pointing}")
            continue
            
        # Load F150W data  
        f150w_path = os.path.join(catalog_dir, f"f150dropout_f150w_catalog.cat")
        f150w_data = read_sextractor_catalog(f150w_path)
        if f150w_data is None:
            print(f"  Missing F150W data for {pointing}")
            continue
        
        # Process each Brenjit_ID
        for src_id in ids:
            # Find source in F115W catalog
            f115w_idx = np.where(f115w_data['NUMBER'] == src_id)[0]
            if len(f115w_idx) == 0:
                print(f"  Source {src_id} not found in F115W catalog")
                continue
                
            # Find source in F150W catalog
            f150w_idx = np.where(f150w_data['NUMBER'] == src_id)[0]
            if len(f150w_idx) == 0:
                print(f"  Source {src_id} not found in F150W catalog")
                continue
            
            # Extract data
            f115w_mag = f115w_data['MAG_AUTO'][f115w_idx[0]]
            f150w_mag = f150w_data['MAG_AUTO'][f150w_idx[0]]
            f115w_snr = f115w_data['SNR'][f115w_idx[0]]
            
            # Calculate color
            color_f115w_f150w = f115w_mag - f150w_mag
            
            # Store results
            results.append({
                'pointing': pointing,
                'ID': src_id,
                'F115w-F150w': color_f115w_f150w,
                'SNRf115w': f115w_snr
            })
            
            print(f"  Source {src_id}: F115W-F150W = {color_f115w_f150w:.3f}, SNR_F115W = {f115w_snr:.1f}")
    
    # Write results to file
    if results:
        with open(output_file, 'w') as f:
            # Write header
            f.write("pointing\tID\tF115w-F150w\tSNRf115w\n")
            
            # Write data
            for result in results:
                f.write(f"{result['pointing']}\t{result['ID']}\t{result['F115w-F150w']:.3f}\t{result['SNRf115w']:.1f}\n")
        
        print(f"\nResults saved to: {output_file}")
        print(f"Total sources processed: {len(results)}")
        
        # Print summary
        colors = [r['F115w-F150w'] for r in results]
        snrs = [r['SNRf115w'] for r in results]
        print(f"F115W-F150W color range: {min(colors):.3f} to {max(colors):.3f}")
        print(f"F115W SNR range: {min(snrs):.1f} to {max(snrs):.1f}")
        
    else:
        print("No data found for any sources.")

if __name__ == '__main__':
    main()

Processing 54 Brenjit_ID sources...

Processing nircam1...
  Source 2858: F115W-F150W = 0.948, SNR_F115W = 12.3
  Source 4010: F115W-F150W = 1.956, SNR_F115W = 11.4
  Source 6802: F115W-F150W = 1.089, SNR_F115W = 13.5
  Source 8216: F115W-F150W = 4.466, SNR_F115W = 0.5
  Source 11572: F115W-F150W = 1.452, SNR_F115W = 8.5
  Source 10899: F115W-F150W = 2.345, SNR_F115W = 2.3
  Source 8272: F115W-F150W = -1.977, SNR_F115W = 8.0

Processing nircam2...
  Source 1332: F115W-F150W = 0.709, SNR_F115W = 15.3
  Source 2034: F115W-F150W = 1.000, SNR_F115W = 15.8
  Source 5726: F115W-F150W = 1.785, SNR_F115W = 5.1
  Source 11316: F115W-F150W = 0.530, SNR_F115W = 7.1

Processing nircam3...
  Source 9992: F115W-F150W = 1.829, SNR_F115W = 3.6

Processing nircam4...
  Source 315: F115W-F150W = 1.312, SNR_F115W = 11.4
  Source 1669: F115W-F150W = 1.382, SNR_F115W = 8.8
  Source 1843: F115W-F150W = 1.052, SNR_F115W = 11.6
  Source 9859: F115W-F150W = 0.312, SNR_F115W = 27.0
  Source 9427: F115W-F150W = 